In [48]:
import requests
from urllib.parse import unquote

saved_file = 'asterix_char1.json'
use_category = True

#base_url = 'https://lotr.fandom.com'
#base_url = 'https://harrypotter.fandom.com'
#base_url = 'https://asterix.fandom.com'
#base_url = 'https://thehungergames.fandom.com'
base_url = 'https://asterix.fandom.com'
#base_url = 'https://narnia.fandom.com'


result = requests.get(base_url+'/sitemap-newsitemapxml-index.xml')

c = result.content
print(c)

b'<?xml version="1.0" encoding="UTF-8"?>\n<sitemapindex xmlns="http://www.sitemaps.org/schemas/sitemap/0.9">\n<sitemap><loc>https://asterix.fandom.com/sitemap-newsitemapxml-NS_0-id-1461-3675.xml</loc></sitemap>\n<sitemap><loc>https://asterix.fandom.com/sitemap-newsitemapxml-NS_14-id-1509-3220.xml</loc></sitemap>\n<sitemap><loc>https://services.fandom.com/discussions-sitemap/sitemap/3118</loc></sitemap>\n</sitemapindex>\n<!-- Generation time: 5ms -->\n<!-- Generation date: 2021-10-14T01:05:46Z -->\n'


In [49]:
from bs4 import BeautifulSoup 
import lxml
import xml.etree.ElementTree as ET
root = ET.fromstring(c)
for link in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
    print(link.text)

https://asterix.fandom.com/sitemap-newsitemapxml-NS_0-id-1461-3675.xml
https://asterix.fandom.com/sitemap-newsitemapxml-NS_14-id-1509-3220.xml
https://services.fandom.com/discussions-sitemap/sitemap/3118


In [50]:
elements = dict()
for page in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
    result = requests.get(page.text)
    c = result.content
    new_root = ET.fromstring(c)
    for element in new_root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
        elements[element.text.split('/')[-1]] = element.text
print('Found {} elements'.format(len(elements)))

Found 337 elements


In [51]:
#elements.items()

In [52]:
characters_dict = {}
if use_category:
    counter=0
    count = 0
    for k,v in elements.items():
        count+=1
        result = requests.get(v)
        c = result.content
        soup = BeautifulSoup(c, "html.parser")  # parse HTML page 
        links = soup.find_all("div", class_='page-header__categories')
        #links2 = soup.find_all("a", class_='page-header__categories')https://lotr.fandom.com/wiki/Category:Characters_in_The_History_of_Middle-earth
        #print(links)
        try:
            if '/wiki/Category:Characters' in str(links[0]) or '/wiki/Category:Female_characters' in str(links[0]) or '/wiki/Category:Male_characters' in str(links[0]):
                if ('Category:' in k or k== 'List_of_Asterix_characters'):
                    continue
            #if '/wiki/Cat%C3%A9gorie:Personnages' in str(links[0]):
                characters_dict[k] = v
                counter+=1
                if counter in range(0,10000,300):
                    print('%s added to the dict, dict len: %s' %(k,str(len(characters_dict))))
        except:
            continue
else:
    characters_dict = elements

In [53]:
print(characters_dict.items())

dict_items([('Asterix_(Character)', 'https://asterix.fandom.com/wiki/Asterix_(Character)'), ('Impedimenta', 'https://asterix.fandom.com/wiki/Impedimenta'), ('Bacteria', 'https://asterix.fandom.com/wiki/Bacteria'), ('The_Pirates', 'https://asterix.fandom.com/wiki/The_Pirates'), ('Odius_Asparagus', 'https://asterix.fandom.com/wiki/Odius_Asparagus'), ('Overanxius', 'https://asterix.fandom.com/wiki/Overanxius'), ('Gracchus_Armisurplus', 'https://asterix.fandom.com/wiki/Gracchus_Armisurplus'), ('Crismus_Bonus', 'https://asterix.fandom.com/wiki/Crismus_Bonus'), ('Surplus_Dairiprodus', 'https://asterix.fandom.com/wiki/Surplus_Dairiprodus'), ('Metallurgix', 'https://asterix.fandom.com/wiki/Metallurgix'), ('Cantankerus', 'https://asterix.fandom.com/wiki/Cantankerus'), ('Redbeard', 'https://asterix.fandom.com/wiki/Redbeard'), ('Metric', 'https://asterix.fandom.com/wiki/Metric'), ('Cleopatra', 'https://asterix.fandom.com/wiki/Cleopatra'), ('Alcoholix', 'https://asterix.fandom.com/wiki/Alcoholix')

In [54]:
import itertools
inverted_dict = {v.encode('utf-8'): k for k, v in characters_dict.items()}
dict(itertools.islice(inverted_dict.items(), 0, 4))

{b'https://asterix.fandom.com/wiki/Asterix_(Character)': 'Asterix_(Character)',
 b'https://asterix.fandom.com/wiki/Impedimenta': 'Impedimenta',
 b'https://asterix.fandom.com/wiki/Bacteria': 'Bacteria',
 b'https://asterix.fandom.com/wiki/The_Pirates': 'The_Pirates'}

In [55]:
import collections
from lxml import html
import pandas as pd
character_count=collections.Counter()
character_text=dict()
character_network = collections.Counter()

characters_details=[]
for character_url, character_name in inverted_dict.items():
    character_html = requests.get(character_url)
    page = html.fromstring(character_html.text)
    soup = BeautifulSoup(character_html.content, "html.parser")
    if (character_name == "Asterix_(Character)"):
        character_name = "Asterix"    

    character_name = unquote(character_name)

    character = {'name':character_name, 'url':character_url, 'html_content':character_html.text }    

    c = []
    s = soup.find('div',{'id':'mw-content-text'})
    if (s is None):
        continue
    for link in s.find_all('a'):
        #print(base_url+str(link.get('href').encode('utf-8')))
        if (link.get('href')) is None:
            continue
        l = (base_url+str(link.get('href').encode('utf-8')))
        d = (base_url+str(link.get('href')))
        if link.get('href') == "/wiki/Asterix" and d.encode('UTF-8') and "https://asterix.fandom.com/wiki/Asterix" not in c:
            print("on ajoute")
            c.append("https://asterix.fandom.com/wiki/Asterix_(Character)")
        if link.get('href') and d.encode('UTF-8') in inverted_dict.keys() and d not in c:
            c.append(d)
    #c = [inverted_dict[base_url+link.get('href').encode('utf-8')] for link in soup.find('div',{'id':'mw-content-text'}).find_all('a') if link.get('href') and (base_url+str(link.get('href').encode('utf-8'))) in inverted_dict]
    lotr_dict={'character_name':character_name,'character_url': character_url.decode("utf-8"),'links':c}
    #lotr_name = soup.find("h2", class_="pi-item pi-item-spacing pi-title")
    characters_details.append(lotr_dict)
print(len(characters_details))

on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
on ajoute
147


In [56]:
import json
with open(saved_file, 'w') as outfile:
    json.dump(characters_details, outfile)